# Reliability Organized

In [9]:
import os,sys
import numpy as np
import pandas as pd
import warnings; warnings.filterwarnings('ignore');

src_dir = os.path.abspath(os.pardir)
sys.path[0] = src_dir

from analysis.CodingScoring import scoreTIPI, normTIPI, stratTIPI, getCronbachAlpha

#### Import data

In [5]:
questions = pd.read_csv('../data/External/dataHeadings.csv')
dataRaw = pd.read_csv('../data/External/dataRaw.csv', names=questions.QuestionCode)
# dataRaw[['E+','A-','C+','S-','O+','E-','A+','C-','S+','O-']].head()

#### Code and score individual items

In [6]:
df = dataRaw[['E+','A-','C+','S-','O+','E-','A+','C-','S+','O-']].replace(['Disagree strongly','Disagree moderately',
                            'Disagree slightly','Neither agree nor disagree',
                            'Agree slightly','Agree moderately','Agree strongly'],
                            [1,2,3,4,5,6,7]).dropna()
#  reverse code
df[['A-','S-','E-','C-','O-']] = 8-df[['A-','S-','E-','C-','O-']]
# df.head()

#### Calculate Cronbach's Alpha

In [4]:
for trait in ['S','E','O','A','C']:
    traitdf = df[[trait+'+',trait+'-']]
    traitT = np.asarray(traitdf).transpose()
    a = getCronbachAlpha(traitT)
    print('%s alpha = %f'% (trait,a))

S alpha = 0.724448
E alpha = 0.792720
O alpha = 0.417748
A alpha = 0.406742
C alpha = 0.656733


Import TIPI norms

In [11]:
TIPInorms = pd.read_csv('../data/External/TIPINorms.csv', index_col=0)
TIPInorms.columns = ['E','A','C','S','O']

Group variables, code and recaculate Cronbach's alpha

In [12]:
for trait in ['S','E','O','A','C']:
    traitdf = df[[trait+'+',trait+'-']]
    highcut = TIPInorms[trait].loc['Mean']+TIPInorms[trait].loc['SD']
    lowcut = TIPInorms[trait].loc['Mean']-TIPInorms[trait].loc['SD']
    traitdf[traitdf>highcut] = 'high'
    traitdf[traitdf[traitdf.applymap(np.isreal)]<lowcut] = 'low'
    traitdf[traitdf.applymap(np.isreal)] = 'mid'
    traitdf = traitdf.replace(['high','mid','low'],[3,2,1])
    traitT = np.asarray(traitdf).transpose()
    a = getCronbachAlpha(traitT)
    print('%s alpha = %f'% (trait,a))

S alpha = 0.671105
E alpha = 0.739151
O alpha = 0.417499
A alpha = 0.371246
C alpha = 0.594727
